In [1]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Flatten, Dense, Dropout, BatchNormalization
from keras.models import Sequential
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox

import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, DirectoryIterator
#to display heatmaps
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

In [2]:
model = Sequential(
[
    keras.Input((128,128,3)),
    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(16, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(8, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(4, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(1, activation='sigmoid')
]
)
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True) #from_logits=True means output probabilities are not normalized
acc_metric = keras.metrics.BinaryAccuracy()
val_acc_metric = keras.metrics.BinaryAccuracy()

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      2432      
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        25632     
_________________________________________________________________
batch_normalization_1 (Batch (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0

In [3]:
@tf.function
def test_step(x):
    val_preds = model(x, training=False)
    # Update val metrics
#     val_acc_metric.update_state(y, val_preds)
    return val_preds

In [4]:
model.load_weights('data_aug_off_model.h5')

In [169]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'imgs/',
    target_size=(128, 128),
    batch_size=13,
    shuffle=True,
    class_mode='binary')

recent_test_batch = test_generator.next()

x_test_batch = recent_test_batch[0]
y_test_batch = recent_test_batch[1]
# print(x_test_batch)
# print(y_test_batch)

pred = test_step(x_test_batch)
# print(val_acc_metric.result())
# val_acc_metric.reset_states()
# print(pred)
for i in range(13):
    print(y_test_batch[i])
    print(np.reshape(tf.get_static_value(pred),(1,13))[0][i])


Found 16 images belonging to 2 classes.
0.0
0.0003042882
0.0
0.99997485
0.0
0.18762308
1.0
0.99966526
1.0
0.69126165
0.0
1.0781934e-15
1.0
1.0
1.0
0.9587014
1.0
0.00011307242
1.0
8.4545296e-05
0.0
0.016766252
1.0
0.9882288
0.0
0.027745582


In [5]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array



def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [7]:
def put_text(imgPath, actual, pred):
    fig, ax = plt.subplots()
    ax.set_xlim(0, 100)
    ax.set_ylim(-30, 100)
    
    img = mpimg.imread(imgPath)
    img = OffsetImage(img, zoom=0.4)
    ab = AnnotationBbox(img, (50, 60))
    ax.add_artist(ab)
    
    plt.text(11, -8, "Ground Truth: {}".format(imgPath[0:-5]), fontsize = 14)
    plt.axis('off')
    
    drowsy = (1 - pred)*100
    
#     Rectangle(xy, width, height
    plt.text(11, -18, "Predicted: {:.2f}% drowsy".format(drowsy), fontsize = 14)
    size = 40
    rect1 = matplotlib.patches.Rectangle((12,-20),size*(1-pred),-5, color = 'red')
    rect2 = matplotlib.patches.Rectangle((12+(size*(1-pred)),-20),size*pred,-5, color = 'green')
    
    greenRect = matplotlib.patches.Rectangle((70,-5),2,2, color = 'green')
    plt.text(74, -5, ": Non-drowsy", fontsize=8)
    redRect = matplotlib.patches.Rectangle((70,-12),2,2, color = 'red')
    plt.text(74,-12, ": Drowsy", fontsize = 8)

    ax.add_patch(rect1)
    ax.add_patch(rect2)
    ax.add_patch(greenRect)
    ax.add_patch(redRect)
    
    plt.savefig(imgPath)

    plt.close()

In [12]:
def save_and_display_gradcam(img_path,actual,pred, heatmap, alpha=0.4):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    
    imgPath = img_path.partition('/')[2].partition('\\')[2].partition('\\')[2]
    superimposed_img.save(imgPath)
    superimposed_img = put_text(imgPath, actual,pred)
    

    # Display Grad CAM
#     display(Image(cam_path))

In [13]:
class ImageWithNames(DirectoryIterator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.filenames_np = np.array(self.filepaths)
        self.class_mode = None # so that we only get the images back

    def _get_batches_of_transformed_samples(self, index_array):
#         print(super()._get_batches_of_transformed_samples(index_array))
        return (super()._get_batches_of_transformed_samples(index_array),
                self.filenames_np[index_array])

In [14]:
# Prepare image
# img_array = preprocess_input(get_img_array("imgs/drowsy/drowsy1", size=(1,128,128,3)))

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = ImageWithNames(
    'testimgs/glasses_bright',test_datagen,
    target_size=(128, 128),
    batch_size=6,
#     shuffle=True,
    class_mode='binary')
x_test_batch, fileNames = next(test_generator)
# print(recent_test_batch)
# x_test_batch = recent_test_batch[0]
# y_test_batch = recent_test_batch[1]

for i in range(len(x_test_batch)):

    # Print what the top predicted class is
    preds = test_step(np.expand_dims(x_test_batch[i],0))
    pred = np.reshape(tf.get_static_value(preds),(1,1))[0][0]
    print(pred)
    
    
    # Remove last layer's softmax
    model.layers[-1].activation = None
    
    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(tf.expand_dims(x_test_batch[i],0), model, 'conv2d_4')

    # Display heatmap
#     plt.matshow(heatmap)
#     plt.show()
    print(fileNames[i])
    actual = '0.0'
    if 'non-drowsy' in fileNames[i]:
        actual = '1.0'
    save_and_display_gradcam(fileNames[i],actual, pred, heatmap)


Found 6 images belonging to 2 classes.
0.9957847
testimgs/glasses_bright\non-drowsy\non-drowsy1.jpg
0.23847169
testimgs/glasses_bright\non-drowsy\non-drowsy3.jpg
0.99999857
testimgs/glasses_bright\non-drowsy\non-drowsy2.jpg
3.8995992e-08
testimgs/glasses_bright\drowsy\drowsy2.jpg
2.3722813e-14
testimgs/glasses_bright\drowsy\drowsy3.jpg
0.9894382
testimgs/glasses_bright\drowsy\drowsy1.jpg
